<a href="https://colab.research.google.com/github/JerryChenz/Stock_screener/blob/master/Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd

# --- Configuration ---
MIN_MARKET_CAP = 500e6 # \$500 million
RISK_FREE_RATE = 0.0485 # Example: 4.85%
EQUITY_RISK_PREMIUM = 0.05 # Example: 5.0%
MIN_DP_THRESHOLD = max(RISK_FREE_RATE, 0.02)
MIN_EP_THRESHOLD = RISK_FREE_RATE + EQUITY_RISK_PREMIUM
MAX_DEBT_MARKET_CAP_RATIO = 2.0 # 200%
INDUSTRY_LEVERAGE_MULTIPLIER = 1.25 # 25% above median

In [ ]:
# --- Get Ticker List (Example: Manually define or load from a file/screener) ---
tickers = ["AAPL", "MSFT", "GOOGL", "JPM", "XOM", "NVDA", "META", "TSLA", "..."] # Add many more

# --- Load Prepared Damodaran Data ---
# Assumes you have a CSV/Excel like: 'industry_leverage.csv'
# Columns: 'Industry', 'Median_Liab_Assets_Ratio'
try:
    damodaran_data = pd.read_csv('industry_leverage.csv').set_index('Industry')
    # Normalize industry names if needed
except FileNotFoundError:
    print("ERROR: Damodaran industry leverage file not found.")
    damodaran_data = pd.DataFrame() # Empty df to avoid errors later, but checks will fail

# --- Screening Logic ---
results = []
for ticker_symbol in tickers:
    try:
        print(f"Processing {ticker_symbol}...")
        stock = yf.Ticker(ticker_symbol)
        info = stock.info

        # Basic Filters
        market_cap = info.get('marketCap')
        if not market_cap or market_cap < MIN_MARKET_CAP:
            continue

        price = info.get('regularMarketPrice')
        if not price: price = info.get('currentPrice') # Fallback
        if not price: continue

        # D/P Calculation
        div_yield = info.get('dividendYield') # Already a ratio
        if not div_yield: div_yield = 0.0
        dp_ratio = div_yield

        # E/P Calculation
        eps = info.get('trailingEps')
        if not eps or eps <= 0 or price == 0:
            continue # Skip stocks with no positive earnings
        ep_ratio = eps / price

        # Debt Proxy & Leverage Calculations
        balance_sheet = stock.quarterly_balance_sheet # Or .balance_sheet for annual
        if balance_sheet.empty or len(balance_sheet.columns) == 0:
            continue # Skip if no balance sheet data

        # Get most recent data column
        latest_bs = balance_sheet.iloc[:, 0]

        total_liabilities = latest_bs.get('Total Liab')
        total_assets = latest_bs.get('Total Assets')

        if total_liabilities is None or total_assets is None or market_cap == 0 or total_assets == 0:
            continue # Skip if essential BS data missing

        debt_market_cap_ratio = total_liabilities / market_cap
        company_liab_assets_ratio = total_liabilities / total_assets

        # Apply Filters
        if dp_ratio < MIN_DP_THRESHOLD: continue
        if ep_ratio < MIN_EP_THRESHOLD: continue
        if debt_market_cap_ratio > MAX_DEBT_MARKET_CAP_RATIO: continue

        # Industry Leverage Comparison
        industry = info.get('industry')
        sector = info.get('sector') # Use industry or sector for matching
        relative_leverage_risk = "Normal/Low"
        industry_median_leverage = None

        if not damodaran_data.empty and industry:
            # Attempt to match (requires careful normalization of names)
            if industry in damodaran_data.index:
                 industry_median_leverage = damodaran_data.loc[industry, 'Median_Liab_Assets_Ratio']
            # Add fallback logic using sector if industry fails etc.

            if industry_median_leverage is not None:
                 if company_liab_assets_ratio > (industry_median_leverage * INDUSTRY_LEVERAGE_MULTIPLIER):
                     relative_leverage_risk = "High"
                     # Optionally filter out high risk here:
                     # continue
            else:
                 relative_leverage_risk = "Industry Data Missing"


        # Store results if passes all checks
        results.append({
            'Ticker': ticker_symbol,
            'Name': info.get('shortName', 'N/A'),
            'Industry': industry,
            'Market Cap': market_cap,
            'Price': price,
            'D/P (%)': dp_ratio * 100,
            'E/P (%)': ep_ratio * 100,
            'Total Liab / Mkt Cap (%)': debt_market_cap_ratio * 100,
            'Company Liab / Assets (%)': company_liab_assets_ratio * 100,
            'Industry Median Liab / Assets (%)': (industry_median_leverage * 100) if industry_median_leverage is not None else 'N/A',
            'Relative Leverage Risk': relative_leverage_risk
        })

    except Exception as e:
        print(f"  Error processing {ticker_symbol}: {e}")
        continue # Skip to next ticker on error

# --- Rank and Output ---
if results:
    final_df = pd.DataFrame(results)
    # Filter out High Risk if not done above
    # final_df = final_df[final_df['Relative Leverage Risk'] != "High"]

    # Rank (Example: prioritize E/P, then D/P)
    final_df = final_df.sort_values(by=['E/P (%)', 'D/P (%)'], ascending=[False, False])

    print("\n--- Screening Results ---")
    print(final_df.to_string())
    # Optional: Save to CSV
    # final_df.to_csv("stock_screen_results.csv", index=False)
else:
    print("\nNo stocks passed the screening criteria.")